In [ ]:
import torch
print(f"The preinstalled torch version is: {torch.__version__}")
version_cuda = torch.version.cuda
print(f"The preinstalled CUDA version is: {torch.version.cuda}")

## Install Prerequisites

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
# if version_cuda >= '11.0':
#     !pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html --upgrade -q
# else:
#     !pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html --upgrade -q
# !pip install torchtext==0.9.0 --upgrade -q

# !pip install fastai==2.3.1 -q
# !pip install fastcore==1.3.20 -q

# !pip install git+git://github.com/airctic/icevision.git --upgrade -q
# !pip install git+git://github.com/airctic/icedata.git --upgrade -q
    
# if version_cuda >= '11.0':
#     # Use for CUDA 11.0
#     !pip install mmcv-full=="1.3.3" -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html --upgrade -q   
# else:
#     # Use for CUDA 10.x
#     !pip install mmcv-full=="1.3.3" -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.0/index.html --upgrade -q

# !pip install mmdet==2.12.0 --upgrade -q
# !pip install yolov5-icevision --upgrade -q

In [ ]:
!pip install git+git://github.com/airctic/icevision.git#egg=icevision[all] --upgrade -q
!pip install git+git://github.com/airctic/icedata.git -q
!pip install yolov5-icevision --upgrade -q




In [ ]:
# import IPython
# import time

# def restart_ipyhton():
#     app = IPython.Application.instance()
#     app.kernel.do_shutdown(True)  

# print('before restart')
# restart_ipyhton()
# time.sleep(15)
# print('after restart')

## Load Modules

In [ ]:
import pandas as pd
import timm
from timm import *

from fastai.vision.all import *
from fastai.vision.learner import _update_first_layer
from fastai.callback.wandb import *

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns

In [ ]:
from icevision.all import *
from datetime import *
import pytz
from icevision.widgets import *

In [ ]:
import os

device = 'CPU'
if 'TPU_NAME' in os.environ.keys():
    if os.environ['XRT_TPU_CONFIG'] is not None: device = 'TPU'
elif 'CUDA_VERSION' in os.environ.keys():
    if os.environ['CUDA_VERSION'] is not None: device = 'GPU'

print(device)

In [ ]:
path_dcm = Path('/kaggle/input/siim-covid19-detection')
path_image_resized = '/kaggle/input/siim-covid19-512-images-and-metadata/train'
path_image_resize_output = '/kaggle/output'
path_model_save = Path('/kaggle/working/models')

# image_size=512
# bs=4
# N_FOLDS = 3
# set_seed(42)

In [ ]:
class Config:
    seed_val = 111
#     seed_everything(seed_val)
    fold_num = 0
    job = 1
    num_classes = 4
    image_size = 512
    model_arch = "yolo5"
    bs = 16
    num_workers = 0
#     kfold = 5
#     loss_func = CrossEntropyLossFlat() # or LabelSmoothingCrossEntropyFlat()
#     metrics = [error_rate, accuracy, RocAuc(average='macro'), F1Score(average='macro')]
#     job_name = f'{model_arch}_fold{fold_num}_job{job}'
#     print("Job Name:", job_name)

#     wandb_project = 'SIIM_classifier_public'
#     wandb_run_name = job_name
    
    if device=='GPU':
        fp16 = True
    else:
        fp16 = False
    
cfg = Config()

## Create Resized Images for Training Dataset

In [ ]:
from PIL import Image
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
split = 'train'
save_dir = f'/kaggle/working/covid19-imgs-{cfg.image_size}px/{split}/'
os.makedirs(save_dir, exist_ok=True)

In [ ]:
# # use meta data from resized image dataset; need to add resized imageset creation routine using pydicom
# image_id = []
# dim0 = []
# dim1 = []
# splits = []
# save_dir_image = save_dir + 'image/'
# os.makedirs(save_dir_image, exist_ok=True)

# for dirname, _, filenames in tqdm(os.walk(path_dcm / f'{split}')):
#         for file in filenames:
#             # set keep_ratio=True to have original aspect ratio
#             xray = read_xray(os.path.join(dirname, file))
#             im = resize(xray, size=cfg.image_size)  
#             im.save(os.path.join(save_dir_image, file.replace('.dcm', '_image.png')))
#             image_id.append(file.replace('.dcm', ''))
#             dim0.append(xray.shape[0])
#             dim1.append(xray.shape[1])
#             splits.append(split)
# meta_imgs_resized = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})

# # meta_imgs_resized.head()

In [ ]:
# # Save dataframe to CSV with imgs
# meta_imgs_resized.to_csv(f'/kaggle/working/covid19-imgs-{cfg.image_size}px/meta_imgs_resized.csv')


In [ ]:
meta_imgs_resized = pd.read_csv('/kaggle/input/covid19imgs512pxmeta/meta_imgs_resized.csv')
meta_imgs_resized.head()

# Can skip if image size metadata has image_id column
meta_imgs_resized_copy = meta_imgs_resized.copy()
meta_imgs_resized_copy.rename(columns={'StudyInstanceUID': 'image_id'}, inplace=True)
meta_imgs_resized_copy.head(3)

In [ ]:
# # Change column name to 'StudyInstanceUID'
# meta_imgs_resized_copy = meta_imgs_resized.copy()
# meta_imgs_resized_copy.rename(columns={'image_id': 'StudyInstanceUID'}, inplace=True)
# meta_imgs_resized_copy.head(3)


In [ ]:
# # Save dataframe to CSV with imgs
# meta_imgs_resized_copy.to_csv(f'/kaggle/working/covid19-imgs-{cfg.image_size}px/meta_imgs_resized.csv')

In [ ]:
# # Create zip of images and meta csv
# from zipfile import ZipFile
# import os
# from os.path import basename

# with ZipFile ('covid19-imgs-512px.zip', 'w') as zipObj:
#     for folderName, subFolders, filenames, in os.walk(save_dir):
#         for filename in filenames:
#             filePath = os.path.join(folderName, filename)
#             zipObj.write(filePath, basename(filePath))


## Load DataFrames and Merge Metadata

In [ ]:
train_image_df = pd.read_csv(path_dcm / 'train_image_level.csv')
# train_image_df = pd.read_csv(path_dcm / 'train_study_level.csv')
train_df = train_image_df.copy()
train_df.head()

In [ ]:
# Add Kfold split for train/valid
fold = 0

from sklearn.model_selection import GroupKFold
df = train_df

gkf  = GroupKFold(n_splits = 5)
df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(df, groups = df.StudyInstanceUID.tolist())):
    df.loc[val_idx, 'fold'] = fold
    
train_df = df


In [ ]:
# Create image_id colume from id field
train_df[['image_id', 'image_type']] = train_df['id'].str.split('_', 1, expand=True)
train_df.head(3)

In [ ]:
#Merge image size and original metadata frames
train_df = train_df.merge(meta_imgs_resized_copy, on='image_id')
train_df.head()

In [ ]:
train_df['image_path'] = f'/kaggle/input/../input/siim-covid19-512-images-and-metadata/train/' + train_df.id + '.png'
train_df.head()

In [ ]:
def get_scaled_bbox(row, sz):
    x_scale = row.dim1/sz
    y_scale = row.dim0/sz
#     x_scale = row.width/sz
#     y_scale = row.height/sz
    
    rescaled_bboxes = []
    if row.bboxes != None:
        for bbox in row.bboxes:
            rescaled_bboxes.append( np.array(bbox) / np.array([x_scale, y_scale, x_scale, y_scale]))
            
        return rescaled_bboxes
    else:
        return None
    
def get_bboxes(val):
    val = val.split(' ')
    if val[0] != 'none':
        bboxes = []
        for i, e in enumerate(val):
            if i % 6 == 2:
                bboxes.append([float(b) for b in val[i : i+4]])
        return bboxes
    else:
        return None

In [ ]:
train_df['box_label'] = train_df['label'].apply(lambda x: x.split(' ')[0])
train_df['bboxes'] = train_df['label'].apply(lambda x: get_bboxes(x))
train_df['rescaled_bboxes'] = train_df.apply(lambda row: get_scaled_bbox(row, cfg.image_size), axis=1)
train_df['rescaled_bboxes'] = train_df['rescaled_bboxes'].apply(lambda x: [[0, 0,  1, 1]] if x == None else x)
train_df.head(2)

## Dataloader

In [ ]:
classes = ['0. opacity']
labels = ['opacity']

In [ ]:
# Here ObjectDetectionRecord from icevision and added ClassificationLabelsRecordComponent
def ObjectDetectionClassiRecord():
    return BaseRecord(
        (
            FilepathRecordComponent(),
            InstancesLabelsRecordComponent(),
            BBoxesRecordComponent(),
            ClassificationLabelsRecordComponent(),
#             ScoresRecordComponent(task=Task('tabular')),
        )
    )
template_record = ObjectDetectionClassiRecord()

In [ ]:
ImgSize = namedtuple("ImgSize", "width,height")

#custom parser from icevision - added classification class map and classification labels
class CustomParser(Parser):
    def __init__(self, template_record, img_dir, label_df, idmap):
        super().__init__(template_record=template_record)

        self.img_dir = img_dir
        self.df = label_df
        self.class_map = ClassMap(list(self.df['box_label'].unique()))
#         self.classi_class_map = ClassMap(['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance'], background=None)
        self.idmap = idmap

    def __iter__(self):
        for o in self.df.itertuples():
            yield o

    def __len__(self) -> int:
        return len(self.df)

    def record_id(self, o):
        return o.id

    def parse_fields(self, o, record, is_new):
        if is_new:
#             record.set_filepath(f'{self.img_dir}/{o.image_id}.png')
#             record.set_filepath(f'{self.img_dir}/{o.image_id}.jpg')
            record.set_filepath(o.image_path)
            record.set_img_size(ImgSize(width=cfg.image_size, height=cfg.image_size))
            record.detection.set_class_map(self.class_map)
#             record.classification.set_class_map(self.classi_class_map)
#             record.classification.set_labels([o.classi_label])
#             record.tabular.set_scores([o.Modality_n, o.PatientSex_n])

        record.detection.add_bboxes([BBox.from_xyxy(float(bb[0]), float(bb[1]), float(bb[2]), float(bb[3])) for bb in o.rescaled_bboxes])
        record.detection.add_labels([o.box_label for _ in o.rescaled_bboxes])

In [ ]:
idmap = IDMap([o[1].id for o in train_df.iterrows()])
train_idx = [o[1].id for o in train_df[train_df["fold"] != fold].iterrows()]
valid_idx = [o[1].id for o in train_df[train_df["fold"] == fold].iterrows()]
split = FixedSplitter([train_idx, valid_idx])

In [ ]:
class_map = ClassMap(labels)
class_map

In [ ]:
parser = CustomParser(template_record, path_image_resized, train_df, idmap)

In [ ]:
train_records, valid_records = parser.parse(autofix=True, data_splitter=split)

In [ ]:
show_records(train_records[:6], ncols=3, class_map=class_map)

In [ ]:
train_tfms = tfms.A.Adapter([*tfms.A.aug_tfms(size=cfg.image_size), tfms.A.Normalize()])
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(cfg.image_size), tfms.A.Normalize()])

train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

# train_dl = train_dl_m(train_ds, batch_size=bs, num_workers=0, shuffle=True)
# valid_dl = valid_dl_m(valid_ds, batch_size=bs, num_workers=0, shuffle=False)

In [ ]:
extra_args = {}
model_type = models.ultralytics.yolov5
backbone = model_type.backbones.small
# The yolov5 model requires an img_size parameter
extra_args['img_size'] = cfg.image_size

In [ ]:
# Instantiate the mdoel
model = model_type.model(backbone=backbone(pretrained=True), num_classes=len(class_map), **extra_args) 

In [ ]:
# DataLoaders
train_dl = model_type.train_dl(train_ds, batch_size=cfg.bs, num_workers=0, shuffle=True)
valid_dl = model_type.valid_dl(valid_ds, batch_size=cfg.bs, num_workers=0, shuffle=False)

In [ ]:
run_datetime = datetime.now(pytz.timezone('US/Pacific')).strftime("%m%d%Y-%H%M")


model_save_name = backbone.model_name + '_'  
model_save_name += str(len(parser.class_map)) + 'cls_' 
model_save_name += str(cfg.image_size) + 'px_'
model_save_name += str(len(train_ds)) + 'ds_'
model_save_name += run_datetime
print(model_save_name)

In [ ]:
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

In [ ]:
learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)

In [ ]:
# learn.freeze()
learn.lr_find()

In [ ]:
cfg.lr1 = 1e-2
cfg.epochs1 = 2

learn.fit(cfg.epochs1, cfg.lr1)

In [ ]:
from fastai.callback.tracker import SaveModelCallback, CancelStepException
model_save_path = str(path_model_save / model_save_name)
sm = SaveModelCallback(fname=model_save_path)

In [ ]:
cfg.lr1 = 1e-2
cfg.lr2 = 1e-4
cfg.epochs1 = 1

# Save Model
learn.unfreeze()
learn.fit_one_cycle(cfg.epochs1, lr_max=slice(cfg.lr2, cfg.lr1), cbs=sm)

In [ ]:
# # Download model
# import os
# os.chdir(r'/kaggle/working/models')
# from IPython.display import FileLink 
# # FileLink(r'yolov5s_3cls_512px_5068ds_07272021-1530.pth')

# FileLink(r'yolov5s_3cls_512px_5068ds_08022021-1449.pt')

## Save Fastai Model and Load

In [ ]:
model_tst = model_type.model(backbone=backbone(pretrained=True), num_classes=len(class_map), device=torch.device("cuda"), **extra_args) 
model_tst.load_state_dict(torch.load(model_save_path+'.pth'))

In [ ]:
model_type.show_results(model_tst, valid_ds, detection_threshold=0.25)

## Save Pytorch Weights and Load Model


In [ ]:
torch.save(model_tst.state_dict(), model_save_path+'.pt')
model_save_path+'.pt'

In [ ]:
model_tst2 = model_type.model(backbone=backbone(pretrained=True), num_classes=len(class_map), device=torch.device("cuda"), **extra_args) 
model_tst2.load_state_dict(torch.load(model_save_path+'.pt'))

In [ ]:
# Batch Prediction
infer_dl = model_type.infer_dl(valid_ds, batch_size=1)
preds = model_type.predict_from_dl(model=model_tst2, infer_dl=infer_dl, detection_threshold=0.25, keep_images=True)

# preds2 = model_type.convert_raw_predictions(model=model_tst2, infer_dl=infer_dl, detection_threshold=0.25, keep_images=True)

## COVID19 Test Dataset Inference 
### Setup Test Images and Metadata

In [ ]:
df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
if df.shape[0] == 2477:
    fast_sub = True
    fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
                         ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
                         ['65761e66de9f_image', 'none 1 0 0 1 1'], 
                         ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
                       columns=['id', 'PredictionString'])
else:
    fast_sub = False

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:

split = 'test'
save_dir = f'/kaggle/tmp/{split}/'

os.makedirs(save_dir, exist_ok=True)

save_dir = f'/kaggle/tmp/{split}/study/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=600)  
    study = '00086460a852' + '_study.png'
    im.save(os.path.join(save_dir, study))
    xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=600)  
    study = '000c9c05fd14' + '_study.png'
    im.save(os.path.join(save_dir, study))
else:   
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=600)  
            study = dirname.split('/')[-2] + '_study.png'
            im.save(os.path.join(save_dir, study))


In [ ]:
image_id = []
dim0 = []
dim1 = []
splits = []
save_dir = f'/kaggle/tmp/{split}/image/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=512)  
    im.save(os.path.join(save_dir,'65761e66de9f_image.png'))
    image_id.append('65761e66de9f.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
    xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=512)  
    im.save(os.path.join(save_dir, '51759b5579bc_image.png'))
    image_id.append('51759b5579bc.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
else:
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=512)  
            im.save(os.path.join(save_dir, file.replace('.dcm', '_image.png')))
            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            splits.append(split)
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})

In [ ]:
if fast_sub:
    df = fast_df.copy()
else:
    df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
id_laststr_list  = []
for i in range(df.shape[0]):
    id_laststr_list.append(df.loc[i,'id'][-1])
df['id_last_str'] = id_laststr_list

study_len = df[df['id_last_str'] == 'y'].shape[0]

In [ ]:
meta = meta[meta['split'] == 'test']
if fast_sub:
    test_df = fast_df.copy()
else:
    test_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
test_df = df[study_len:].reset_index(drop=True) 
meta['image_id'] = meta['image_id'] + '_image'
meta.columns = ['id', 'dim0', 'dim1', 'split']
test_df = pd.merge(test_df, meta, on = 'id', how = 'left')

In [ ]:
test_df.head()

### Test Images Dataloader

In [ ]:
save_dir

In [ ]:
images = []
filenames = []
for filename in os.listdir(save_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        imgfile = os.path.join(save_dir, filename)
        images.append(open_img(imgfile))
        filenames.append(filename)
    else:
        continue
# images = np.array(images)
# filenames2images = dict(zip(filenames, images))

In [ ]:
infer_tst_ds = Dataset.from_images(images, valid_tfms, class_map)
# infer_tst_ds = Dataset.from_images(images, valid_tfms)

In [ ]:
# Batch Prediction
infer_tst_dl = model_type.infer_dl(infer_tst_ds, batch_size=1)
preds = model_type.predict_from_dl(model=model_tst2, infer_dl=infer_tst_dl, detection_threshold=0.012, keep_images=True)
filenames2preds = dict(zip(filenames, preds))

### Create Yolo5 Prediction Output

In [ ]:
image_ids = []
PredictionStrings = []

for filename in filenames2preds.keys():
    image_id = filename.split('/')[-1].split('.')[0]
    pred = filenames2preds[filename]
    pred_labels = getattr(pred.pred.detection,'labels')
    pred_scores = getattr(pred.pred.detection, 'scores')
    pred_str = ''

    for i, bbox in enumerate(getattr(pred.pred.detection, 'bboxes')):

        label = str(0)
        score = str(pred_scores[i])
        xmin = str(int(float(bbox.xmin)))
        ymin = str(int(float(bbox.ymin)))
        xmax = str(int(float(bbox.xmax)))
        ymax = str(int(float(bbox.ymax)))
        
        pred_str += ' ' + str(label)
        pred_str += ' ' + str(score) 
        pred_str += ' ' + str(xmin) 
        pred_str += ' ' + str(ymin)
        pred_str += ' ' + str(xmax) 
        pred_str += ' ' + str(ymax)
        
    PredictionStrings.append(pred_str) 
    image_ids.append(image_id)

pred_df = pd.DataFrame({'id':image_ids,'PredictionString':PredictionStrings})
pred_df.head()


### Yolo5 Detection

In [ ]:
# for file_path in tqdm(glob('runs/detect/exp/labels/*.txt')):
#     print('file_path=', file_path)
#     image_id = file_path.split('/')[-1].split('.')[0]
#     print('image_id=', image_id)
#     w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
#     f = open(file_path, 'r')
#     data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
#     # Reorder prediction tensor as label, confidence, bbox (x1,y1,x2,y2)
#     data = data[:, [0, 5, 1, 2, 3, 4]]
#     print('data_revised', data)
#     # Modify prediction tensor to voc format
#     bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
#     print('bboxes_beforeloop=', bboxes)
#     # Modify bbox coordinates from float to int
#     for idx in range(len(bboxes)):
#         bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
# #         print('bboxes_loop',idx, '=', bboxes[idx])
#     image_ids.append(image_id)
#     PredictionStrings.append(' '.join(bboxes))


# pred_df = pd.DataFrame({'id':image_ids,'PredictionString':PredictionStrings})